# Tutorial: Math Reasoning

Let's walk through a quick example of setting up a `dspy.ChainOfThought` module and optimizing it for answering algebra questions.

Install the latest DSPy via `pip install -U dspy` and follow along.

Let's tell DSPy that we will use OpenAI's `gpt-4o-mini` in our modules. To authenticate, DSPy will look into your `OPENAI_API_KEY`. You can easily swap this out for [other providers or local models](https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb).

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import dspy


# dspy.configure(lm=gpt4o_mini)  # we'll use gpt-4o-mini as the default LM, unless otherwise specified

/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next, let's load some data examples from the [MATH](https://arxiv.org/abs/2103.03874) benchmark. We'll use a training split for optimization and evaluate it on a held-out dev set.

Please note that the following step will require:
```bash
%pip install git+https://github.com/hendrycks/math.git
```

In [3]:
import dotenv
dotenv.load_dotenv()

True

In [4]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

350 350


Let's inspect one example from the training set.

In [5]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.
Answer: 221


Now let's define our module. It's extremely simple: just a chain-of-thought step that takes a `question` and produces an `answer`.

In [9]:
# with dspy.settings.configure(lm=gpt4o_mini):
module = dspy.ChainOfThought("question -> answer")
#     module(question=example.question)

Next, let's set up an evaluator for the zero-shot module above, before prompt optimization.

In [10]:
THREADS = 35
kwargs = dict(num_threads=THREADS, display_progress=True)
evaluate = dspy.Evaluate(devset=dataset.dev, metric=dataset.metric, **kwargs)

gpt4o_mini = dspy.LM('openai/gpt-4o-mini', max_tokens=2000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=2000)
r1 = dspy.LM('fireworks_ai/accounts/fireworks/models/deepseek-r1', max_tokens=4000)

with dspy.settings.context(lm=gpt4o):
    gpt4o_uncompiled = evaluate(module)

with dspy.settings.context(lm=r1):
    r1_uncompiled = evaluate(module)


NameError: name 'module' is not defined

In [21]:
r1.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `question` (str)\n\nYour output fields are:\n1. `reasoning` (str)\n2. `answer` (str)\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\nInputs will have the following structure:\n\n[[ ## question ## ]]\n{question}\n\nOutputs will be a JSON object with the following fields.\n\n{\n  "reasoning": "{reasoning}",\n  "answer": "{answer}"\n}\n\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.'},
  {'role': 'user',
   'content': '[[ ## question ## ]]\nTwo numbers are independently selected from the set of positive integers less than or equal to 5. What is the probability that the sum of the two numbers is greater than their product? Express your answer as a common fraction.\n\nRespond with a JSON object in the following order of fields: `reasoning`, then `answer`.'}],
 'kwargs': {'tem

And lastly let's optimize our module. Since we want strong reasoning, we'll use the large GPT-4o as the teacher model (used to bootstrap reasoning for the small LM at optimization time) but not as the prompt model (used to craft instructions) or the task model (trained).

GPT-4o will be invoked only a small number of times. The model involved directly in optimization and in the resulting (optimized) program will be GPT-4o-mini.

We will also specify `max_bootstrapped_demos=4` which means we want at most four bootstrapped examples in the prompt and `max_labeled_demos=4` which means that, in total between bootstrapped and pre-labeled examples, we want at most four.

In [29]:
kwargs_4o = dict(num_threads=THREADS, task_model=gpt4o, teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o)
kwargs_r1 = dict(num_threads=THREADS, task_model=r1, teacher_settings=dict(lm=r1), prompt_model=r1)
optimizer_4o = dspy.MIPROv2(metric=dataset.metric, auto="medium", **kwargs_4o)
optimizer_r1 = dspy.MIPROv2(metric=dataset.metric, auto="medium", **kwargs_r1)


kwargs = dict(requires_permission_to_run=False, max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module_4o = optimizer_4o.compile(module, trainset=dataset.train, **kwargs)
optimized_module_r1 = optimizer_r1.compile(module, trainset=dataset.train, **kwargs)

with dspy.settings.context(lm=gpt4o):
    gpt4o_compiled = evaluate(optimized_module_4o)
with dspy.settings.context(lm=r1):
    r1_compiled = evaluate(optimized_module_r1)

gpt4o_compiled
r1_compiled

2025/02/02 01:23:56 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 280

2025/02/02 01:23:56 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/02/02 01:23:56 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/02/02 01:23:56 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  6%|▌         | 4/70 [00:00<00:00, 723.03it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


  6%|▌         | 4/70 [00:00<00:00, 731.26it/s]


...


Average Metric: 224.00 / 349 (64.2%): 100%|█████████▉| 349/350 [05:50<00:05,  5.09s/it]

2025/02/02 02:12:13 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Find the smallest possible value of $$\\frac{(y-x)^2}{(y-z)(z-x)} + \\frac{(z-y)^2}{(z-x)(x-y)} + \\frac{(x-z)^2}{(x-y)(y-z)},$$ where $x,y,$ and $z$ are distinct real numbers.', 'reasoning': 'Combining all three fractions under a single denominator, the given expression is equal to $$\\frac{(x-y)^3 + (y-z)^3 + (z-x)^3}{(x-y)(y-z)(z-x)}.$$ Consider the numerator as a polynomial in $x$, so that $P(x) = (x-y)^3 + (y-z)^3 + (z-x)^3$ (where we treat $y$ and $z$ as fixed values). It follows that $P(y) = (y-y)^3 + (y-z)^3 + (z-y)^3 = 0$, so $y$ is a root of $P(x) = 0$ and $x-y$ divides into $P(x)$. By symmetry, it follows that $y-z$ and $z-x$ divide into $P(x)$. Since $P$ is a cubic in its variables, it follows that $P = k(x-y)(y-z)(z-x)$, where $k$ is a constant. By either expanding the definition of $P$, or by trying test values (if we take $x = 0, y = -1, z = 1$, we obtain $P = -6 = k \\cdot (-2

Average Metric: 224.00 / 349 (64.2%): 100%|██████████| 350/350 [08:18<00:00,  1.42s/it]

2025/02/02 02:12:13 INFO dspy.evaluate.evaluate: Average Metric: 224.0 / 350 (64.0%)


64.0

In [7]:
# optimized_module_4o.save('gpt4o_compiled.json')
# optimized_module_r1.save('r1_compiled.json')


In [8]:
import matplotlib.pyplot as plt
import numpy as np

plt.clf()

# Data for the bars
models = ['GPT-4o', 'R1']
uncompiled_scores = [gpt4o_uncompiled, r1_uncompiled]
compiled_scores = [gpt4o_compiled, r1_compiled]

x = np.arange(len(models))  # Label locations
width = 0.35  # Width of the bars

# Create bars
plt.bar(x - width/2, uncompiled_scores, width, label='Uncompiled')
plt.bar(x + width/2, compiled_scores, width, label='Compiled')

# Customize the plot
plt.xlabel('Models')
plt.ylabel('Score')
plt.title('R1 vs GPT-4o on MATH with MIPROv2')
plt.xticks(x, models)
plt.legend()

plt.show()

NameError: name 'gpt4o_uncompiled' is not defined

<Figure size 640x480 with 0 Axes>

In [51]:
import dspy

chat_adapter = dspy.ChatAdapter()
with dspy.settings.context(lm=r1, adapter=chat_adapter):
    module = dspy.ChainOfThought("question -> answer")
    module(question=example.question)

[autoreload of dspy.adapters.chat_adapter failed: Traceback (most recent call last):
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/isaac/projects/dspy/.venv/lib/python3.12/site-packag

TypeError: super(type, obj): obj must be an instance or subtype of type

In [7]:
evaluate(optimized_module)

Average Metric: 310.00 / 350 (88.6%): 100%|██████████| 350/350 [01:31<00:00,  3.84it/s]

2024/11/28 18:59:19 INFO dspy.evaluate.evaluate: Average Metric: 310 / 350 (88.6%)


,question,example_reasoning,example_answer,pred_reasoning,pred_answer,method
0,What is the smallest integer value of $c$ such that the function $...,The given function has a domain of all real numbers if and only if...,1,The function \( f(x) = \frac{x^2 + 1}{x^2 - x + c} \) will have a ...,1,✔️ [True]
1,What is the least value of $x$ that is a solution of $|{-x+3}|=7$?,"In order to have $|{-x+3}| = 7$, we must have $-x + 3 = 7$ or $-x ...",-4,The equation \( |{-x+3}|=7 \) implies two possible cases: 1. \(-x ...,-4,✔️ [True]
2,Evaluate $\left\lceil -\frac{7}{4}\right\rceil$.,"$-\frac{7}{4}$ is between $-1$ and $-2$, so $\left\lceil -\frac{7}...",-1,"To evaluate \(\left\lceil -\frac{7}{4}\right\rceil\), we first nee...",-1,✔️ [True]
3,"A triangle has vertices at coordinates $(11,1)$, $(2,3)$ and $(3,7...",We must find the distance between each pair of points by using the...,10,To find the length of the sides of the triangle formed by the vert...,10,✔️ [True]
4,Let $f(x) = x + 2$ and $g(x) = 1/f(x)$. What is $g(f(-3))$?,"First, we find that $f(-3) = (-3) + 2 = -1$. Then, $$g(f(-3)) = g(...",1,"To find \( g(f(-3)) \), we first need to evaluate \( f(-3) \). Usi...",1,✔️ [True]


88.57

Neat. It was pretty straightforward to improve quality from 74% to over 88% on a held-out set here.

That said, for reasoning tasks like this, you will often want to consider more advanced strategies, like:

- A `dspy.ReAct` module with access to a calculator function or `dspy.PythonInterpreter`
- Ensembling multiple optimized prompts with a majority vote (or an Aggregator module) on top


Just to understand what changed, let's view the prompt after optimization.

In [8]:
dspy.inspect_history()





[2024-11-28T18:59:19.176586]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Analyze the `question` provided, and systematically apply mathematical reasoning to derive the `answer`. Ensure to articulate each step of your thought process in the `reasoning` field, detailing how you identify relationships and formulate equations to arrive at the solution.


User message:

[[ ## question ## ]]
If $|4x+2|=10$ and $x<0$, what is the value of $x$?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Assistant 